## CHURN - USE CASE

### Load libraries and functions

In [1]:
import numpy as np
import pandas as pd
import random
import os
import datetime
from collections import Counter

# Split Dataset into Train- Test
from sklearn.model_selection import train_test_split

# Machine Learning Models
import xgboost as xgb
from sklearn.linear_model import LogisticRegression

# Performance Metrics
from sklearn.metrics import confusion_matrix


- [1.- Data](#section1)
    + [1.1.- Read Data](#section1.1)
    + [1.2.- Discuss and prepare target variable](#section1.2)
    + [1.3.- Create additional variables](#section1.3)

- [2.- Model](#section2)
    + [2.1.- Logistic Regression](#section2.1)
    + [2.2.- XGBoost](#section2.2)
    + [2.3.- Evaluation of Models](#section2.3)

In [2]:
# Load auxiliar metrics functions
from aux_metrics import get_uplift, get_feature_importance # you should have this script in the same folder as the notebook

In [4]:
# Set seed for reproducibility
random.seed(42)

# Set data path
project_path = './' # path to indicate where your file is saved in your local computer
# You can put the full path or the relative path, ./ is the same folder as where the notebook is located

<a id='section1'></a>
## STEP 1: Data 

<a id='section1.1'></a>
### Read Data

In [5]:
churn_file_name = "bankingchurn_data_modif.csv"
df_churn = pd.read_csv(
    project_path + churn_file_name, 
    na_values=['-'], 
    sep = ';', # the separator between columns is ;, it can be checked in notepad
    decimal = ',' # to make sure pandas understands that the , is used as decimal seprator and reads numeric columns as numeric
)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Data dimensions (#rows, #columns)
df_churn.shape

(100000, 44)

In [7]:
# Make quick cross-checks
df_churn.head()

,contract_start,contract_end,date_of_birth,gender,profession,size_household,ZIP,segment,credit_rating,no_credit_rating_flag,...,online_number_of_logins_per_month,online_tranactions_per_month,advisor_contacts_last12months,customer_limit,cash_withdraws_per_month,cash_withdrawals_value,consumer_credit_value,consumer_credit_maturity,account_fee,number_of_refusals
0,2010-10-24,NaN,1991-02-11,M,Professional Services,1,76829,S2,89.568518,0,...,0,0,3,1000.0,4,459.840940,-24011.393552,205,15,6
1,2010-04-30,2017-10-20,1986-04-06,W,NaN,1,35104,S2,94.270507,0,...,16,8,2,1000.0,7,476.334459,-5368.743068,137,15,6
2,2013-05-02,NaN,1993-04-18,M,Transportation,1,63450,S2,93.030947,0,...,26,12,2,1000.0,3,332.640202,-9678.713419,624,15,4
3,2016-12-24,NaN,1994-07-29,W,Professional Services,2,9573,S3,NaN,1,...,0,0,4,1050.0,4,378.324744,-12110.305916,1263,10,7
4,2011-10-01,NaN,1990-10-28,W,Research,1,66916,S2,90.457664,0,...,23,18,3,1000.0,4,471.509299,0.000000,521,10,3


In [8]:
df_churn.head().T

,0,1,2,3,4
contract_start,2010-10-24,2010-04-30,2013-05-02,2016-12-24,2011-10-01
contract_end,NaN,2017-10-20,NaN,NaN,NaN
date_of_birth,1991-02-11,1986-04-06,1993-04-18,1994-07-29,1990-10-28
gender,M,W,M,W,W
profession,Professional Services,NaN,Transportation,Professional Services,Research
size_household,1,1,1,2,1
ZIP,76829,35104,63450,9573,66916
segment,S2,S2,S2,S3,S2
credit_rating,89.5685,94.2705,93.0309,NaN,90.4577
no_credit_rating_flag,0,0,0,1,0


In [9]:
# Statistical Information: .describe()
df_churn.describe()

,size_household,credit_rating,no_credit_rating_flag,main_account_flag,online_banking_flag,tele_banking_flag,creditcard_flag,insurance_life_flag,insurance_house_flag,insurance_car_flag,...,online_number_of_logins_per_month,online_tranactions_per_month,advisor_contacts_last12months,customer_limit,cash_withdraws_per_month,cash_withdrawals_value,consumer_credit_value,consumer_credit_maturity,account_fee,number_of_refusals
count,100000.000000,77517.000000,100000.000000,100000.00000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000
mean,1.665850,91.480805,0.224830,0.48612,0.729260,0.35982,0.479610,0.222810,1.076250,0.710510,...,14.194810,9.18393,1.654470,1287.443365,4.036680,300.769580,-3394.078264,456.426170,10.51490,5.043130
std,0.793788,5.668550,0.417472,0.49981,0.444344,0.47995,0.499587,0.416134,0.265399,0.453528,...,12.112868,10.04345,1.807432,512.328403,1.716183,189.729464,5959.513473,485.093374,6.09149,4.241062
min,1.000000,43.720633,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.00000,0.000000,50.000000,0.000000,0.000000,-43213.432348,0.000000,0.00000,0.000000
25%,1.000000,88.680536,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.00000,0.000000,1000.000000,3.000000,154.226137,-5075.175558,0.000000,10.00000,1.000000
50%,1.000000,92.778025,0.000000,0.00000,1.000000,0.00000,0.000000,0.000000,1.000000,1.000000,...,15.000000,6.00000,1.000000,1000.000000,4.000000,295.055682,0.000000,302.000000,10.00000,5.000000
75%,2.000000,95.653020,0.000000,1.00000,1.000000,1.00000,1.000000,0.000000,1.000000,1.000000,...,24.000000,16.00000,3.000000,1500.000000,5.000000,431.438064,0.000000,858.000000,15.00000,8.000000
max,6.000000,99.495558,1.000000,1.00000,1.000000,1.00000,1.000000,1.000000,2.000000,1.000000,...,67.000000,59.00000,6.000000,4800.000000,8.000000,1172.363497,0.000000,3003.000000,20.00000,31.000000


In [10]:
# Info about nulls: .info()
print(df_churn.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 44 columns):
contract_start                       100000 non-null object
contract_end                         4871 non-null object
date_of_birth                        100000 non-null object
gender                               100000 non-null object
profession                           92271 non-null object
size_household                       100000 non-null int64
ZIP                                  100000 non-null object
segment                              100000 non-null object
credit_rating                        77517 non-null float64
no_credit_rating_flag                100000 non-null int64
main_account_flag                    100000 non-null int64
online_banking_flag                  100000 non-null int64
tele_banking_flag                    100000 non-null int64
creditcard_flag                      100000 non-null int64
insurance_life_flag                  100000 non-null int64


In [11]:
print(Counter(df_churn.dtypes))

Counter({dtype('int64'): 21, dtype('float64'): 16, dtype('O'): 7})


In [12]:
for col in df_churn.columns: 
    print(col, df_churn[col].nunique())

contract_start 8232
contract_end 365
date_of_birth 14842
gender 2
profession 48
size_household 6
ZIP 16055
segment 5
credit_rating 77517
no_credit_rating_flag 2
main_account_flag 2
online_banking_flag 2
tele_banking_flag 2
creditcard_flag 2
insurance_life_flag 2
insurance_house_flag 2
insurance_car_flag 2
insurance_other_flag 2
mortgage_flag 2
portfolio_flag 2
last_balance 100000
last_balance_minus_6_months 100000
last_balance_minus_12_months 100000
income_salary_per_year 37003
income_deposits_per_year 19340
income_securities_per_year 19346
insurance_life_premium_per_month 10809
insurance_house_premium_per_year 45150
insurance_car_premium_per_year 34535
mortgage_value 16677
mortgage_interest 20779
mortgage_downpayment 20795
mortgage_dayuntilmaturity 8589
mortgage_numbers 4
online_number_of_logins_per_month 62
online_tranactions_per_month 57
advisor_contacts_last12months 7
customer_limit 9833
cash_withdraws_per_month 9
cash_withdrawals_value 93358
consumer_credit_value 36993
consumer_cr

<a id='section1.2'></a>
### Discuss and prepare target variable

In [13]:
## Task 1b: Create target, 1 means churn 0 means no churn
df_churn["target"] = np.where(df_churn['contract_end'].isna(), 0,1)

In [14]:
# Check distribution of the target variable
df_churn.groupby('target').size()/df_churn.shape[0]

target
0    0.95129
1    0.04871
dtype: float64

<a id='section1.3'></a>
### Create Additional Variables

In [19]:
df_churn['age'] = (pd.Timestamp.today() - \
                   pd.to_datetime(df_churn['date_of_birth']))/np.timedelta64(1, 'Y')
df_churn[['contract_start', 'date_of_birth', 'age']].head()

,contract_start,date_of_birth,age
0,2010-10-24,1991-02-11,28.692717
1,2010-04-30,1986-04-06,33.544289
2,2013-05-02,1993-04-18,26.510605
3,2016-12-24,1994-07-29,25.232003
4,2011-10-01,1990-10-28,28.982935


In [20]:
# remove variables not needed (dates, helper veriables, row index...)
df_churn = df_churn.drop(['contract_start', 'contract_end',  'date_of_birth', 'ZIP'], axis = 1)

In [21]:
# Missing data imputation - Replace each missing value with an own category, e.g. "NA -> Not_available"
df_churn.loc[df_churn['profession'].isna(), 'profession'] = 'unknown'

<a id='section2'></a>
### STEP 2: MODEL

- Separate the dataset into train - dev - test
- Build a model 
- Obtain model performance


<a id='section2.1'></a>
### Logistic Regression

In [22]:
# Option 1: Logistic regression with manual variable selection

logit_model = LogisticRegression()
var_select  = ["size_household", 
               "income_deposits_per_year", 
               "main_account_flag", 
               "insurance_house_premium_per_year", 
               "income_securities_per_year", 
               "cash_withdraws_per_month"]



In [24]:
# Split data: leave out evaluation set

# X: explanatory variables, y: target
X = df_churn[var_select]
y = df_churn['target']

    # 1. Train (train + test), evaluation
x_train, x_eval_logit, y_train, y_eval_logit = train_test_split(X, y, test_size=0.2)

    # 2. Split training data again into training and test set
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# 2.1. Training set: fit the model
logit_model.fit(x_train,y_train)

# 2.2. Test set: 
# Predictions
predictions_logreg = logit_model.predict(x_test)

# Check accuracy of predictions
accuracy_logreg = confusion_matrix(y_test.values, predictions_logreg)

print(sum(np.diagonal(accuracy_logreg))/sum(sum(accuracy_logreg)))
print(accuracy_logreg)
print(get_uplift(predictions_logreg, y_test.values)[['percentile','acc_uplift']].head())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9519375
[[15231    12]
 [  757     0]]
        percentile  acc_uplift
n_tile                        
1.0              1    0.000000
2.0              2    0.000000
3.0              3    0.000000
4.0              4    0.000000
5.0              5    1.666667


C:\Users\Vuk Vuksanovic\Documents\_recruiting\analytics\20191016_upm_event\aux_metrics.py:21: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  uplift_result = uplift_df.groupby(["n_tile"])["y_true"].agg({'num_observations':'count','num_positives':"sum", "segment_precision":"mean"})


<a id='section2.2'></a>
### XGBoost

In [25]:
# Option 2: Xgboost without variable selection (all numerical variables are included)

# Specify the kind of model to develop
xgb_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                          colsample_bytree = 0.3, 
                          learning_rate = 0.1,
                          max_depth = 5, 
                          n_estimators = 350)

In [27]:
# Split data: leave out evaluation set
    # Variable selection: return columns in df_churn.columns that are not in []
var_select = np.setdiff1d(df_churn.columns, ['target', 'profession', 'segment'])

    # X: explanatory variables, y: target
x_train = df_churn[var_select].apply(pd.to_numeric, errors='coerce')
y_train = df_churn['target']

    # 1. Train (train + test), evaluation
x_train, x_eval_xgb, y_train, y_eval_xgb = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

    # 2. Split training data again into training and test set
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# 2.1. Training set: fit the model
xgb_reg.fit(x_train, y_train)

# 2.2. Test set: 
# Predictions
preds = xgb_reg.predict(x_test)

# Check accuracy of predictions
accuracy_xgb = confusion_matrix(y_test.values, np.round(preds))
    
print(sum(np.diagonal(accuracy_xgb))/sum(sum(accuracy_xgb)))
print(accuracy_xgb)

0.9696875
[[15144    50]
 [  435   371]]


In [28]:
# Obtain the most important variables
scores_xgb = xgb_reg.get_booster().get_score(importance_type='gain')

most_imp_xgb = pd.DataFrame({'feature':list(scores_xgb.keys()), 'gain':list(scores_xgb.values())}).sort_values(by = ['gain'], ascending = False)
most_imp_xgb['gain'] = most_imp_xgb['gain']/most_imp_xgb['gain'].sum()
most_impt_xgb = most_imp_xgb.reset_index().drop('index', axis =1)
print(most_imp_xgb.head())


                            feature      gain
7                    size_household  0.094653
9          income_deposits_per_year  0.071581
0  insurance_house_premium_per_year  0.064344
4        income_securities_per_year  0.062961
1               online_banking_flag  0.059390


<a id='section2.3'></a>
### Evaluation of Models

In [29]:
# Evaluation of your model (TASK 4)---------------------------------------------------------------
# Evaluate your best model on the top decile of the evaluation dataset

# Option 1: Linear regression 

# Validation set:
# Predictions
preds_eval_logit  = logit_model.predict(x_eval_logit)
# Performance Metrics
accuracy_logit_eval = confusion_matrix(y_eval_logit.values, preds_eval_logit)
print(sum(np.diagonal(accuracy_logit_eval))/sum(sum(accuracy_logit_eval)))
print(get_uplift(preds_eval_logit, y_eval_logit.values)[['percentile','acc_uplift']].head())

# Option 2: Xgboost

# Validation set
# Predictions
preds_eval_xgb = xgb_reg.predict(x_eval_xgb)

# Performance Metrics
accuracy_xgb_eval = confusion_matrix(y_eval_xgb.values, np.round(preds_eval_xgb))
sum(np.diagonal(accuracy_xgb_eval))/sum(sum(accuracy_xgb_eval))
print(get_uplift(preds_eval_xgb, y_eval_xgb.values)[['percentile','acc_uplift']].head())

0.9509
        percentile  acc_uplift
n_tile                        
1.0              1         0.0
2.0              2         0.0
3.0              3         0.0
4.0              4         0.0
5.0              5         0.0


C:\Users\Vuk Vuksanovic\Documents\_recruiting\analytics\20191016_upm_event\aux_metrics.py:21: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  uplift_result = uplift_df.groupby(["n_tile"])["y_true"].agg({'num_observations':'count','num_positives':"sum", "segment_precision":"mean"})


        percentile  acc_uplift
n_tile                        
1.0              1   11.563523
2.0              2   11.043258
3.0              3   11.244218
4.0              4   11.213637
5.0              5   10.966897
